<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<p><p><p><p>
<b>Rubin Image Cutout Service Tutorial</b> <br>
Contact author: <i>Leanne Guy</i> <br>
Last verified to run: <i>2022-04-29</i> <br>
LSST Science Piplines version: Weekly <i>2022_17</i> <br>
Container Size: <i>medium</i> <br>
Targeted learning level: <i>intermediate</i> <br>

In [ ]:
# Import general python packages
import numpy as np
import re
import pandas
from pandas.testing import assert_frame_equal
import uuid
import requests
import warnings

# Import LSST packages
from lsst.rsp import get_tap_service, retrieve_query
from lsst.rsp.utils import get_access_token
import lsst.daf.butler as Butler
import lsst.geom as geom
import lsst.resources
from lsst.afw.image.exposure import Exposure, ExposureF


# PyVO
import pyvo
from pyvo.dal.adhoc import DatalinkResults, SodaQuery
from typing import Optional
import pyvo.auth.authsession
import requests

# Plotting with MPL
import matplotlib.pyplot as plt
import lsst.afw.display as afwDisplay
import lsst.afw.image as afwImage


# Astropy
from astropy import units as u
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy.utils.data import download_file
from astropy.wcs import WCS       
from astropy.visualization import simple_norm, imshow_norm
from astropy.visualization import ImageNormalize,  ZScaleInterval, AsinhStretch
from astropy.units import UnitsWarning

# Holoviz for interactive visualization
import bokeh
from bokeh.io import output_file, output_notebook, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
import holoviews as hv
from holoviews import streams, opts
from holoviews.operation.datashader import rasterize

# Set the holoviews plotting library to be bokeh
# You will see the holoviews + bokeh icons displayed when the library is loaded successfully
hv.extension('bokeh')

# Display bokeh plots inline in the notebook
output_notebook()

# AFW backend to MPL
afwDisplay.setDefaultBackend('matplotlib')


In [ ]:
# Ignore warnings
warnings.simplefilter("ignore", category=UnitsWarning)
warnings.simplefilter("ignore", category=UserWarning)

# Set the maximum number of rows to display from pandas
pandas.set_option('display.max_rows', 20)

In [ ]:
# This should match the verified version listed at the start of the notebook
! echo ${IMAGE_DESCRIPTION}
! eups list lsst_distrib

In [ ]:
# DP0.2 Butler repository 
config = 'dp02'
collection = '2.2i/runs/DP0.2'
butler = Butler.Butler(config, collections=collection)
registry = butler.registry

In [ ]:
# Calexp
datasetType = 'calexp'
dataId = {'visit': 192350, 'detector': 175}
calexp = butler.get(datasetType, dataId=dataId)
ci = calexp.getInfo()

In [ ]:
fig = plt.figure()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(calexp.image)
plt.title(f'PVI:{ci.getId()}, {ci.getFilter()}')
plt.show()

## 1. Define and get a cutout with the Butler

In [ ]:
# Coordinates of a region on this image
x,y = (310, 2095)
wcs = calexp.getWcs()
radec = wcs.pixelToSky(x,y)
print(radec.getRa().asDegrees(), radec.getDec().asDegrees())

In [ ]:
# Define a circle around the point
point = lsst.geom.SpherePoint(
    radec.getRa().asDegrees() * lsst.geom.degrees, 
    radec.getDec().asDegrees() * lsst.geom.degrees)
radius = 10 * lsst.geom.arcseconds
point, radius

In [ ]:
# Need the UUID for the cutout service 
datasetRef = registry.findDataset('calexp', dataId)
calexp_uuid = datasetRef.id
assert isinstance(calexp_uuid, uuid.UUID)
calexp_uuid

In [ ]:
# Use the Butler to get the cutout
cutoutSideLength = 100
cutoutSize = geom.ExtentI(cutoutSideLength, cutoutSideLength)
xy = geom.PointI(x,y)
bbox = geom.BoxI(xy - cutoutSize // 2, cutoutSize)
parameters = {'bbox': bbox}
dsType = "calexp"

In [ ]:
cutout_image = butler.get(dsType, parameters=parameters, dataId=dataId)
assert cutout_image is not None
print("The size of the cutout in pixels is: ", cutout_image.image.array.shape)

In [ ]:
fig = plt.figure()
display = afwDisplay.Display(frame=fig)
display.scale('asinh', 'zscale')
display.mtv(cutout_image.image)
plt.title(f'PVI:{ci.getId()}, {ci.getFilter()}')
plt.show()

### 2. Querying the ObsCore table for a data product (calexp)

In [ ]:
# PyVo : # https://pyvo.readthedocs.io/en/latest/api/pyvo.dal.SIAService.html
# TODO provide an introduction to SODA in the description section
import pyvo as vo
from pyvo.dal.adhoc import DatalinkResults, SodaQuery

In [ ]:
service = get_tap_service()
#service.describe()
# Get the image from# Get our RSP access token (we will need this to download the data)
token = get_access_token()
# token

In [ ]:
query = "SELECT COUNT(*) from ivoa.ObsCore"
result = service.search(query).to_table()
result

The ivoa.ObsCore contains 8475974 entries

In [ ]:
# Query the ObsCore table for the image usig the UUID 
# The UUID is part of the access url field 
# IS THERE A BETTER WAY TO DO THIS?
url_str = '%' + str(datasetRef.id) + '%'
query = """SELECT * FROM ivoa.ObsCore  
WHERE access_url like '""" + url_str + """'
"""
print(query)
results = service.search(query)

In [ ]:
# We passed a UUID so there must be 1 result only
assert len(results) == 1  
results.to_table().show_in_notebook()

The access_url in the LSST ObsCore model is is a link to a DataLink links service. It does not link directly to the image itself. This service provides access to the image but also to other information. Let's extract the access URL and look at it

In [ ]:
# Extract the datalinks access url - this gives a lot of information about a data product
result = results[0]
f"Datalink links service url: {result.getdataurl()}"

In [ ]:
# Now we can use the datalinks service to access the (transient) signed URL for image access
dr = DatalinkResults.from_result_url(result.getdataurl(),session=service._session)
print(dr.status)

In [ ]:
# Note that there are two results, the first is the access URL for the primary data product,
# the second is the SODA cutout serice
dr.to_table().show_in_notebook()

In [ ]:
# Now we can grab the google signed URL for the image (note that this will expire)
image_url = dr.getrecord(0).get('access_url')
print(image_url)

In [ ]:
# and then download the image and look at the FITS file information - looks like a PVI
image_file = download_file(image_url)
hdulist = fits.open(image_file)
for hdu in hdulist:
    print(hdu.name)
# fits.info(image_file)

In [ ]:
# Let's plot the image and see what it looks like
image = hdulist[1].data

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
im = imshow_norm(image, ax, origin='lower', 
                 interval=ZScaleInterval(), 
                 stretch=AsinhStretch(), cmap='gray')
fig.colorbar(im[0])

In [ ]:
# This is the same image as we retrieved via the Butler

## 3. Using the Image Cutout Service 

The second result in the DatalinkResults gives us the SODAcutout serivce

In [ ]:
# Create a Sodaquery Serivce 
sq = SodaQuery.from_resource(dr, dr.get_adhocservice_by_id("cutout-sync"), session=service._session)

In [ ]:
# Now define a circular cutout region using the ra, dec and radius
ra = radec.getRa().asDegrees()
dec = radec.getDec().asDegrees()
print(ra, dec)
radius = 0.09 # units
sq.circle = (ra, dec, radius)

In [ ]:
# Execute the cuout and save to a file
sodaPoly = os.path.join(os.getenv('HOME'), 'soda-polygon.fits')
with open(sodaPoly, 'bw') as f:
    f.write(sq.execute_stream().read())

In [ ]:
# Display the cutout
i = lsst.afw.image.ImageF(sodaPoly)     #read FITS file into afw image object
afw_display = lsst.afw.display.Display()      #get an alias to the lsst.afw.display.Display() method
afw_display.scale('asinh', 'zscale')    #set the image stretch algorithm and range
afw_display.mtv(i)                     #load the image into the display

# Circle type defined in DALI
https://www.ivoa.net/documents/DALI/20170517/REC-DALI-1.1.html#tth_sEc3.3.6

Circle values serialised in VOTable or service parameters must have the following metadata in the FIELD element: datatype="double" or datatype="float", arraysize="3", xtype="circle". For circles in a spherical coordinate system, the values are ordered as: longitude latitude radius; longitude values must fall within [0,360], latitude values within [-90,90], and radius values in (0,180]. For example:

12.3 45.6 0.5

In spherical coordinates, all longitude values must fall within [0,360] and all latitude values within [-90,90].

In [ ]:
# Gregory How do I call the cutout service to give me the same cutout as I get above from the Butler? 
calexp_uuid = datasetRef.id
cutout_coords = SkyCoord(radec.getRa().asDegrees()*u.degree, radec.getDec().asDegrees()*u.degree, unit="deg", frame="icrs")
cutout_radius = 10
cutout_pos = 'CIRCLE 55.8 -32.3 10.0'
cutout_pos

In [ ]:
query = SodaQuery(ics, id=calexp_uuid, pos = cutout_pos)